# Project Plan

- Write a web scraper that scrapes title, description, metrics etc. on a list of marvel superheroes from Wikipedia
- Uses PyMYSQL to write to a database 
	- Several tables 
- Data Cleaning with Python and SQL
- Calculated fields with Python SQL

Eventually:

- Data Modeling in either PowerBI or SQL itself
- Visualization that looks very professional and fun


# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
import csv
import pymysql
import re

pd.set_option("display.max_rows", None)

# Table 1: Character Scraper

In [2]:
# https://en.wikipedia.org/wiki/Characters_of_the_Marvel_Cinematic_Universe

'''class Character(self, name, designation, description, link):
    self.name = name
    self.designation = designation
    self.description = description
    self.link = link
'''
def get_major_characters():
    html = requests.get('https://en.wikipedia.org/wiki/Characters_of_the_Marvel_Cinematic_Universe')
    bs = BeautifulSoup(html.text, 'lxml')
    
    #Character Names
    try:
        names = bs.find_all('h3')
        names = [names[i].getText().rstrip('[edit]') for i in range(len(names)-2) if not names[i].getText().startswith('Introduced')]
    except:
        print('There was an error pulling character names.')
    
    #Character Descriptions
    descriptions = []
    try:
        for heading in bs.find_all('h3'):
            description = heading.findNext('p')
            if not description.getText().startswith('The depiction of adapted and original characters in the MCU'):
                descriptions.append(description.getText())
    except:
        print('There was an error in pulling descriptions.')
    
    #Movie Appearances
    appearances = []
    try: 
        for heading in bs.find_all('h3'):
            appearance = heading.find_previous_sibling('p')
            if not appearance.getText().startswith('The following is a supplementary list of characters that appear in lesser roles') and not appearance.getText().startswith('Phase Four'):
                appearances.append(appearance.getText())
    except:
        print('There was an error in pulling appearances.')
        
    try:
        last_appearance = bs.find('span',id = 'Minor_characters').parent.find_previous_sibling('p').getText()
        appearances.append(last_appearance)
    except:
        print('Could not pull final appearance.')
        
    #Designation (find the previous h2)
    designations = []
    try:
        ref_name = bs.find_all('h3')
        for item in ref_name:
            if not item.getText().startswith('Introduced') and not item.getText() == 'Search':
                designation = item.find_previous_sibling('h2').getText()
                designations.append(designation)
    except:
        print('There was an error in pulling designations.')
                
    return names, designations, descriptions, appearances


In [3]:
names, designations, descriptions, appearances = get_major_characters()

There was an error in pulling descriptions.
There was an error in pulling appearances.
Could not pull final appearance.
There was an error in pulling designations.


In [4]:
print(f'There are {len(names)} in names')
print(f'There are {len(descriptions)} in descriptions')
print(f'There are {len(appearances)} in appearances')
print(f'There are {len(designations)} in designations')

There are 0 in names
There are 0 in descriptions
There are 0 in appearances
There are 0 in designations


In [5]:
character_data = list(zip(names, designations, descriptions, appearances))

In [6]:
character_df = pd.DataFrame(character_data, columns = ['Name','Designation','Description','Appearances'])

In [7]:
character_data_dict = dict((z[0],list(z[1:])) for z in zip(names, designations,descriptions, appearances))

# Write To Characters Table

In [8]:
'''conn = pymysql.connect(host = '****',user = 'root', passwd = '*****', db = 'mysql', 
                       charset = 'utf8')
cur = conn.cursor()
cur.execute('USE marvel_database')

def store_characters(name, designation, description, appearance):
    cur.execute('INSERT INTO characters (char_name,char_type,char_desc,appearances) VALUES ''("%s","%s","%s","%s")',
                (name, designation, description, appearance))
    cur.connection.commit()
try:
    for key, value in character_data_dict.items():
        store_characters(key,value[0],value[1],value[2])
finally:
    cur.close()
    conn.close()
'''

'conn = pymysql.connect(host = \'127.0.0.1\',user = \'root\', passwd = \'Ipgatt77\', db = \'mysql\', \n                       charset = \'utf8\')\ncur = conn.cursor()\ncur.execute(\'USE marvel_database\')\n\ndef store_characters(name, designation, description, appearance):\n    cur.execute(\'INSERT INTO characters (char_name,char_type,char_desc,appearances) VALUES \'\'("%s","%s","%s","%s")\',\n                (name, designation, description, appearance))\n    cur.connection.commit()\ntry:\n    for key, value in character_data_dict.items():\n        store_characters(key,value[0],value[1],value[2])\nfinally:\n    cur.close()\n    conn.close()\n'

Fields I might want to add to this table:
- Number of Appearances (calculation off of appearances column)
- Country
- Debut Year

I also might want to go back and scrape minor characters as well. 

# Scraping Character Details

In [9]:
# Grab the link from the characters table for each actor
# Grab the portrayed by: information
# Grab the actor link from those
# Take the relevant details from that page

In [10]:
def get_links():
    html = requests.get('https://en.wikipedia.org/wiki/Characters_of_the_Marvel_Cinematic_Universe')
    bs = BeautifulSoup(html.text, 'lxml')
    
    # No links available for Darcy Lewis and Katy
    links = []
    try:
        div = bs.find_all('div', {'role':'note'})
        for item in div:
            link = item.findNext('a')['href']
            if link == '/wiki/Marvel_Cinematic_Universe:_Phase_One':
                break
            else:
                links.append(link)
    except:
        print('There was an error retrieiving links.')
    
    links = links[1:]
    return links

In [11]:
links = get_links()

In [12]:
def get_character(link_list):
    
    '''
    Errors occur when the link redirects to a list of characters rather than the main character page
    '''
    master_list = []
    for item in link_list:
        html = requests.get(f'https://en.wikipedia.org{item}')
        bs = BeautifulSoup(html.text,'lxml')
        
        try:
             #Get category headers
            portrayed = bs.find('table', {'class':'infobox'})
            categories = portrayed.find_all('th',{'class':'infobox-label'})
            category_list = [i.getText() for i in categories]

            #Get category data
            data = portrayed.find_all('td',{'class':'infobox-data'})
            data_list = [i.getText() for i in data]

            #Get name
            char_name = portrayed.find('th').getText()
            label = 'char_name'

            category_list.append(label)                
            data_list.append(char_name)

        except:
            print(f'There was an error in retrieving the character information for: {item}.')

        data_dict = dict(zip(category_list,data_list))
        master_list.append(data_dict)
            
    return master_list

In [13]:
character_info = get_character(links)

In [14]:
cleaned_entries = []
for item in character_info:
    new_dict = {f"{key.lower().replace(' ','_').replace('(s)','')}": val for key, val in item.items()}
    cleaned_entries.append(new_dict)

In [16]:
'''conn = pymysql.connect(host = '127.0.0.1',user = 'root', passwd = 'Ipgatt77', db = 'mysql', 
                       charset = 'utf8')
cur = conn.cursor()
cur.execute('USE marvel_database')

def store_data(char_list):
    for item in char_list:
        columns = ', '.join(item.keys())
        values = list(item.values())
        values = [i.replace('"',"") for i in values]
        values = ', '.join(['"'+str(i)+'"' for i in values])
        cur.execute(f'INSERT INTO character_details ({columns}) VALUES ({values});')
        cur.connection.commit()

try:
    store_data(cleaned_entries)
finally:
    cur.close()
    conn.close()
'''

'conn = pymysql.connect(host = \'127.0.0.1\',user = \'root\', passwd = \'Ipgatt77\', db = \'mysql\', \n                       charset = \'utf8\')\ncur = conn.cursor()\ncur.execute(\'USE marvel_database\')\n\ndef store_data(char_list):\n    for item in char_list:\n        columns = \', \'.join(item.keys())\n        values = list(item.values())\n        values = [i.replace(\'"\',"") for i in values]\n        values = \', \'.join([\'"\'+str(i)+\'"\' for i in values])\n        cur.execute(f\'INSERT INTO character_details ({columns}) VALUES ({values});\')\n        cur.connection.commit()\n\ntry:\n    store_data(cleaned_entries)\nfinally:\n    cur.close()\n    conn.close()\n'

## Clean Character Details

In [17]:
char_details = pd.read_csv('C:/Users/Nickf/OneDrive/Documents/Data Analysis/Portfolio Project 6 - Marvel Webscraper/char_details.csv')

In [18]:
#Find the percentage of missing values by column
char_details_preprocessed = char_details.copy()

missing_values = char_details.isna().mean()*100
columns_to_drop = missing_values[missing_values > missing_values.mean()].index

char_details_preprocessed = char_details_preprocessed.drop(columns_to_drop, axis = 1)
char_details_preprocessed = char_details_preprocessed.dropna(axis = 0,subset = ['char_name'])

char_details_preprocessed = char_details_preprocessed.reset_index().drop('index', axis = 1)

char_details_preprocessed

,character_details_id,first_appearance,based_on,adapted_by,portrayed_by,occupation,affiliation,weapon,nationality,char_name,full_name,created_by,species,publisher,alter_ego,team_affiliations,notable_aliases,abilities
0,1,The Incredible Hulk (2008),Hulkby Stan LeeJack Kirby,Zak Penn,\nEdward Norton (2008; The Incredible Hulk)\nM...,\nAvenger\nGladiator\nBiochemist\nNuclear phys...,\nAvengers\nCulver University\nRevengers\nS.H....,\nHulkbuster armor\nNano Gauntlet\n,American,Bruce Banner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Captain America: The First Avenger (2011),Bucky Barnesby Joe SimonJack Kirby,Christopher MarkusStephen McFeely,Sebastian Stan,\nAssassin\nSoldier\n,\nHydra\nSoviet Union\nHowling Commandos\nStra...,\nMetal prosthetic arms[1]\nCaptain America's ...,American,Bucky Barnes,James Buchanan Bucky Barnes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Thor (2011),Hawkeyeby Stan LeeDon Heck,\nJ. Michael Straczynski\nMark Protosevich\nAs...,Jeremy Renner,\nAvenger\nVigilante\nAgent of S.H.I.E.L.D.\n,\nAvengers\nS.H.I.E.L.D.\nS.T.R.I.K.E.[1]\n,\nBow and quiver; Compound and Recurve bows[2]...,American,Clint Barton,Clinton Francis Barton,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Captain America: The First Avenger (2011),Peggy Carterby Stan LeeJack Kirby,Christopher MarkusStephen McFeely,\nHayley Atwell\nGabriella Graves (young)\n,\nSpecial agent\nDirector of S.H.I.E.L.D.\n,\nS.H.I.E.L.D.\nStrategic Scientific Reserve\n...,NaN,British,Peggy Carter,Margaret Carter,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Iron Man (2008),NaN,NaN,Clark Gregg,S.H.I.E.L.D. agentS.H.I.E.L.D. director,S.H.I.E.L.D.,NaN,American,Phil Coulson,Phillip J. Coulson[1],Mark FergusHawk OstbyArt MarcumMatt Holloway,\nHuman\nLife Model Decoy\n,NaN,NaN,NaN,NaN,NaN
5,6,Captain Marvel (2019),Carol Danversby Roy ThomasGene Colan,\nAnna Boden\nRyan Fleck\nGeneva Robertson-Dwo...,\nBrie Larson\nMckenna Grace (13 years old)\nL...,\nAvenger[b]\nStarforce warrior\nFighter pilot\n,\nAvengers\nSkrulls\nKree Empire\nStarforce\nU...,NaN,NaN,Carol Danvers,NaN,NaN,Human–Kree hybrid,NaN,NaN,NaN,NaN,NaN
6,7,Guardians of the Galaxy (2014),Drax the Destroyerby Jim Starlin,\nJames Gunn\nNicole Perlman\n,Dave Bautista,NaN,Guardians of the Galaxy,\nDual knives\nVarious weapons including the H...,NaN,Drax,Drax,NaN,Unknown,NaN,NaN,NaN,NaN,NaN
7,8,Iron Man (2008),\nNick Furyby Stan Lee Jack Kirby\nUltimate...,\nMark FergusHawk Ostby\nArt MarcumMatt Hollow...,Samuel L. Jackson,\nDirector of S.H.I.E.L.D.\nAgent of S.H.I.E.L...,\nAvengers\nCentral Intelligence Agency\nS.H.I...,NaN,American,Nick Fury,Nicholas Joseph Fury,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,Guardians of the Galaxy (2014),Gamoraby Jim Starlin,\nJames Gunn\nNicole Perlman\n,\nZoe Saldaña\nAriana Greenblatt (young)\n,\nGuardian\nAssassin\nWarlord (What If...?)\n,\nGuardians of the Galaxy\nChildren of Thanos\...,\nGodslayer sword[4]\nBejeweled knife\nVarious...,NaN,Gamora,Gamora,NaN,Zehoberi[2],NaN,NaN,NaN,NaN,NaN
9,10,Guardians of the Galaxy (2014),Grootby Stan LeeLarry LieberJack Kirby,\nJames Gunn\nNicole Perlman\n,NaN,\nGuardian of the Galaxy\nBounty hunter (Adult...,Guardians of the Galaxy,NaN,NaN,Groot,NaN,NaN,Flora colossus,NaN,NaN,NaN,NaN,NaN


In [19]:
char_details_preprocessed = char_details_preprocessed.fillna('N/A')
based_on_split = char_details_preprocessed['based_on'].str.split('by ', expand = True)
based_on = based_on_split[0]
# To remove unicode spaces and newlines
based_1 = based_on_split[1].str.normalize('NFKD').replace(r'\n',' ', regex = True)
based_2 = based_on_split[2].str.normalize('NFKD').replace(r'\n',' ', regex = True)
based_3 = based_on_split[3].str.normalize('NFKD').replace(r'\n',' ', regex = True)

# For simplicity we are just going to keep based (source material) and based_1 (author)

char_details_preprocessed['based_on_source'] = based_on
char_details_preprocessed['based_on_author'] = based_1
char_details_preprocessed = char_details_preprocessed.fillna('N/A')
char_details_preprocessed = char_details_preprocessed.drop(['based_on'], axis = 1)

In [20]:
# There are quite a few newline characters 
# On first examination, they look like they should all be replaced with a comma or semi-colon with the exception of the first occurence
char_details_preprocessed = char_details_preprocessed.replace(r'\n',', ',regex = True)
df_obj = char_details_preprocessed.select_dtypes(['object'])

char_details_preprocessed[df_obj.columns] = df_obj.apply(lambda x: x.str.lstrip(', ').str.rstrip(', '))

In [21]:
# To fix spacing issues
def separate_values(value):
    for i in range(0,len(value)):
        try:
            for i in range(0,len(value)):
                if value[i].islower() and value[i+1].isupper():
                    value = value[:i+1] + ', ' + value[i+1:]
                else:
                    pass
        except:
            continue
    return value

In [44]:
columns_to_fix = ['based_on_author','adapted_by','occupation','created_by','team_affiliations','notable_aliases','abilities']
for item in columns_to_fix:
    char_details_preprocessed[item] = char_details_preprocessed[item].apply(lambda row: separate_values(row))
char_details_preprocessed['adapted_by'] = char_details_preprocessed['adapted_by'].apply(lambda x: x.replace('Stephen Mc, Feely','Stephen McFeely'))
len(char_details_preprocessed.columns)

19

In [81]:
row_list = []
for index, row in char_details_preprocessed.iterrows():
    row_list.append(list(row.values))
    
cleaned_rows = []
for row in row_list:
    row = map(str,row)
    cleaned_rows.append('("' + '", "'.join(row) + '")')


("1", "The Incredible Hulk (2008)", "Zak Penn", "Edward Norton (2008; The Incredible Hulk), Mark Ruffalo (2012–present)", "Avenger, Gladiator, Biochemist, Nuclear physicist, Professor, Consultant for S.H.I.E.L.D.", "Avengers, Culver University, Revengers, S.H.I.E.L.D.", "Hulkbuster armor, Nano Gauntlet", "American", "Bruce Banner", "N/A", "N/A", "N/A", "N/A", "N/A", "N/A", "N/A", "N/A", "Hulk", "Stan Lee, Jack Kirby")
("2", "Captain America:  The First Avenger (2011)", "Christopher Markus, Stephen McFeely", "Sebastian Stan", "Assassin, Soldier", "Hydra, Soviet Union, Howling Commandos, Strategic Scientific Reserve, United States Army", "Metal prosthetic arms[1], Captain America's shield, Various firearms, Grenade launcher", "American", "Bucky Barnes", "James Buchanan Bucky Barnes", "N/A", "N/A", "N/A", "N/A", "N/A", "N/A", "N/A", "Bucky Barnes", "Joe Simon, Jack Kirby")
("3", "Thor (2011)", "J. Michael Straczynski, Mark Protosevich, Ashley Miller, Zack Stentz, Don Payne", "Jeremy Renne

In [87]:
'''conn = pymysql.connect(host = '******',user = '*****', passwd = '****', db = 'mysql', 
                       charset = 'utf8')
cur = conn.cursor()
cur.execute('USE marvel_database')

def store_data_frame(df):
    columns = list(char_details_preprocessed.columns)
    columns = ', '.join(columns)
    
    row_list = []
    for index, row in char_details_preprocessed.iterrows():
        row_list.append(list(row.values))
    
    cleaned_rows = []
    for row in row_list:
        row = map(str,row)
        cleaned_rows.append('("' + '", "'.join(row) + '")')
                            
    final_rows = ', '.join(cleaned_rows)
    
    print(f'INSERT INTO character_details_clean ({columns}) VALUES ({final_rows});')
    cur.execute(f'INSERT INTO character_details_clean ({columns}) VALUES {final_rows};')
    cur.connection.commit()

try:
    store_data_frame(char_details_preprocessed)
finally:
    cur.close()
    conn.close()'''

INSERT INTO character_details_clean (character_details_id, first_appearance, adapted_by, portrayed_by, occupation, affiliation, weapon, nationality, char_name, full_name, created_by, species, publisher, alter_ego, team_affiliations, notable_aliases, abilities, based_on_source, based_on_author) VALUES (("1", "The Incredible Hulk (2008)", "Zak Penn", "Edward Norton (2008; The Incredible Hulk), Mark Ruffalo (2012–present)", "Avenger, Gladiator, Biochemist, Nuclear physicist, Professor, Consultant for S.H.I.E.L.D.", "Avengers, Culver University, Revengers, S.H.I.E.L.D.", "Hulkbuster armor, Nano Gauntlet", "American", "Bruce Banner", "N/A", "N/A", "N/A", "N/A", "N/A", "N/A", "N/A", "N/A", "Hulk", "Stan Lee, Jack Kirby"), ("2", "Captain America:  The First Avenger (2011)", "Christopher Markus, Stephen McFeely", "Sebastian Stan", "Assassin, Soldier", "Hydra, Soviet Union, Howling Commandos, Strategic Scientific Reserve, United States Army", "Metal prosthetic arms[1], Captain America's shield,